In [16]:
import sys
import os

sys.path.append(os.path.abspath(".."))  # Move one level up to 'project_root'
from scripts.utilities import *

import pandas as pd
import numpy as np
from linearmodels.panel import PanelOLS
from statsmodels.stats.sandwich_covariance import cov_hac
import statsmodels.api as sm
from scipy import stats
from tqdm import tqdm

In [17]:
R = pd.read_csv('../data/returns.csv', index_col=0).iloc[:100]
R

,INCR,EURN,NRP,AUB,ROIV,RBA,SBGI,EBON,JCTCF,EXAS,...,PFC,GM,BOTJ,GROV,DRH,ODD,SITM,CPBI,VRTS,TEL
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.358566,0.009836,0.010309,0.100001,-0.042308,0.020711,-0.020725,-0.100000,0.000000,-0.041667,...,0.000000,0.002048,0.000000,0.015228,0.075472,0.051967,0.434615,-0.005495,-0.160952,-0.001287
3,-0.120235,-0.017857,-0.002551,-0.011364,-0.006024,0.037681,0.005291,0.051111,-0.031250,-0.021739,...,0.009091,-0.005254,0.000000,-0.015100,-0.017544,0.051200,-0.032172,0.000000,0.349603,-0.042525
4,-0.033333,0.016529,-0.025576,0.057472,0.001919,-0.027933,0.010526,-0.160677,0.000000,-0.004444,...,-0.009009,-0.024355,0.000000,0.000000,0.000000,-0.054985,-0.049861,0.002210,-0.138772,-0.012113
5,0.000000,-0.051219,0.004725,0.000000,0.008166,0.048850,-0.020833,0.100756,0.032258,0.000000,...,0.018182,0.006918,0.090909,0.000000,-0.013393,-0.027783,-0.008163,-0.007718,-0.032227,-0.020436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0.000000,0.004008,0.004349,-0.062761,0.000000,-0.004386,-0.029126,-0.108508,0.000000,-0.015018,...,0.008849,0.000000,0.000000,0.001028,-0.009523,0.069542,-0.043127,0.000000,0.014706,0.020178
97,0.000000,-0.015968,0.004764,0.017857,0.001019,-0.030837,0.015000,0.098202,0.000000,0.125561,...,-0.052632,-0.017037,0.000000,0.001541,-0.012238,-0.021125,-0.012676,0.002927,-0.026465,0.018034
98,-0.150000,-0.002704,0.007759,-0.017543,0.000000,-0.031818,-0.044335,-0.080605,0.000000,-0.066932,...,0.000000,-0.024451,0.000000,-0.004103,0.005309,-0.004765,-0.063718,-0.005837,-0.019417,0.019143
99,0.029412,-0.009492,-0.014115,0.071428,0.004073,-0.014084,0.000000,0.008219,0.000000,0.031597,...,-0.027778,-0.023794,-0.023809,0.001030,0.014965,0.063081,0.057389,0.001957,0.019142,-0.014017


# Cross-Validation

The first approach is to shuffle the dataset multiple times to make train/test folds, then construct the models for mean and variance on the train subsample and assess the results on the test subsample.  
Test size will be 10%

In [18]:
def get_subsamples(data: pd.DataFrame, train_size : float = 0.7, shuffle: bool = True, random_state: int = None):

    if shuffle: data = data.sample(frac=1, axis=1, random_state=random_state)   
    data_train = data.iloc[:, :int(data.shape[1] * train_size)]
    data_test = data.iloc[:, int(data.shape[1] * train_size):]

    return data_train, data_test


### Pooled Diebold–Mariano Test for Equal Predictive Accuracy

Let $L_m$ denote the pooled average squared-error loss for forecasting model $m$, computed over $n$ cross-sectional units and $T$ time periods:

$$
L_m = \frac{1}{nT} \sum_{t=1}^{T} \sum_{i=1}^{n} \left( y_{i,t} - \hat{y}_{i,t}^{(m)} \right)^2
$$

#### Hypotheses

- **Null Hypothesis ($H_0$):** The expected pooled average loss is equal for both models:
  $$
  H_0: \mathbb{E}[L_{m_1}] = \mathbb{E}[L_{m_2}]
  $$

- **Alternative Hypothesis ($H_1$):** The expected pooled average loss of the unrestricted model is smaller:
  $$
  H_1: \mathbb{E}[L_{m_1}] > \mathbb{E}[L_{m_2}]
  $$

where:
- $m_2$ is unrestricted model: $R_t = \beta_1 + \gamma_t + u_{it}$
- $m_1$ is restricted model: $R_t = \beta_2 + v_{it}$

#### Test Statistic

Define the loss differential for unit $i$ at time $t$ as:
$$
\Delta L_{i,t} = \left( y_{i,t} - \hat{y}_{i,t}^{(m_1)} \right)^2 - \left( y_{i,t} - \hat{y}_{i,t}^{(m_2)} \right)^2
$$

Compute the cross-sectional average loss differential at each time $t$:
$$
\Delta L_t = \frac{1}{n} \sum_{i=1}^{n} \Delta L_{i,t}
$$

Scale these to obtain:
$$
\hat{L}_t = \sqrt{n} \cdot \Delta L_t
$$

The test statistic is then:
$$
J_{st}= \frac{1}{\sqrt{T}} \sum_{t=1}^{T} \frac{\hat{L}_t - \bar{{\hat{L}}}}{\hat{\sigma}_{\hat{L}}}
$$

where:
- $\bar{\hat{L}} = \frac{1}{T} \sum_{t=1}^{T} \hat{L}_t$ is the average of the $\hat{L}_t$ series,
- $\hat{\sigma}_{\hat{L}}$ is a consistent estimator of the standard deviation of $\hat{L}_t$, which can be computed using the Newey–West estimator to account for potential autocorrelation.

#### Asymptotic Distribution

Under the null hypothesis and standard regularity conditions, the test statistic $J_{st}$ converges in distribution to the standard normal distribution:

$$
J_{st} \xrightarrow{d} \mathcal{N}(0, 1)
$$

Source:
[Allan Timmermann & Yinchu Zhu (2019). Comparing Forecasting Performance with Panel Data. SSRN Working Paper](https://rady.ucsd.edu/_files/faculty-research/timmermann/Panel%20Forecast%20Comparison%20Tests%2004_30_2019.pdf)


#### Newey–West Estimator for $\hat{\sigma}_{\hat{L}}$

To account for potential autocorrelation in the series of scaled average loss differentials $\hat{L}_t$, we employ the Newey–West estimator to obtain a consistent estimate of the standard deviation $\hat{\sigma}_{\hat{L}}$.

Let $\tilde{L}_t = \hat{L}_t - \bar{\hat{L}}$ denote the demeaned series, where $\bar{\hat{L}} = \frac{1}{T} \sum_{t=1}^{T} \hat{L}_t$ is the sample mean of $\hat{L}_t$.

The Newey–West estimator of the variance $\hat{\sigma}_{\hat{L}}^2$ is given by:

$$
\hat{\sigma}_{\hat{L}}^2 = \sum_{j = -J}^{J} \left(1 - \frac{|j|}{J + 1}\right) \hat{\gamma}(j)
$$

where:
- $J$ is the maximum lag length (also known as the bandwidth),
- $\hat{\gamma}(j)$ is the sample autocovariance at lag $j$, defined as:

$$
\hat{\gamma}(j) = \frac{1}{T} \sum_{t = |j| + 1}^{T} \tilde{L}_t \tilde{L}_{t - |j|}
$$

For negative lags ($j < 0$), we set $\hat{\gamma}(j) = \hat{\gamma}(-j)$ to ensure symmetry.

The weights $\left(1 - \frac{|j|}{J + 1}\right)$ correspond to the Bartlett kernel, which assigns decreasing weights to autocovariances at higher lags.

Finally, the standard deviation estimate is obtained by taking the square root of the variance estimate:

$$
\hat{\sigma}_{\hat{L}} = \sqrt{\hat{\sigma}_{\hat{L}}^2}
$$

This estimator provides a heteroskedasticity and autocorrelation consistent (HAC) estimate of the standard deviation, which is crucial for valid inference in the presence of autocorrelated loss differentials.

Source:
[Newey, W. K., & West, K. D. (1987). A simple, positive semi-definite, heteroskedasticity and autocorrelation consistent covariance matrix. Econometrica, 55(3), 703–708](https://users.ssc.wisc.edu/~bhansen/718/NeweyWest1987.pdf?utm_source=chatgpt.com)

#### This test will be used both for Mean and Variance predictions

In [58]:
def pooled_DM(L, n):

    T = len(L)

    # Scale loss differentials
    L_hat = np.sqrt(n) * L.values

    # OLS with intercept only to prepare for HAC variance estimation
    X = np.ones((T, 1))
    ols = sm.OLS(L_hat, X).fit()

    # Newey–West standard error
    nlags = int(np.floor(4 * (T / 100) ** (2 / 9)))
    hac_cov = cov_hac(ols, nlags=nlags)
    se_hac = np.sqrt(hac_cov[0, 0])

    # Compute test statistic and p-value
    J_st = ols.params[0] / se_hac
    p_value = 1 - stats.norm.cdf(J_st)

    return p_value, J_st

def compare_mean(train: pd.DataFrame, test: pd.DataFrame):

    model = PanelOLS(train['ret'], train[['const']], entity_effects=True, time_effects=True).fit(cov_type="robust")
    
    # Extract average time effects to use in test set predictions
    gamma = model.estimated_effects.groupby('t').mean()['estimated_effects']
    test['gamma'] = test.index.get_level_values('t').map(gamma)
    test['pred_model'] = test['gamma'] + model.params['const']
    test['e2_model'] = (test['ret'] - test['pred_model']) ** 2

    # Fit restricted model with only intercept
    model_const = PanelOLS(train['ret'], train[['const']], entity_effects=True, time_effects=False).fit(cov_type="robust")
    test['pred_const'] = model_const.params['const']
    test['e2_const'] = (test['ret'] - test['pred_const']) ** 2

    # Compute average squared error loss differential per time period
    L = (test['e2_const'] - test['e2_model']).groupby('t').mean()
    n = test.index.get_level_values('i').nunique()
    
    p_value, J_st = pooled_DM(L, n)

    return p_value, J_st, model


In [ ]:
def cross_val_mean(data_wide: pd.DataFrame, n: int = 5, fix_random: bool = True, train_size: float = 0.9):
    """
    Perform cross-validation of the means.
    """

    results = []
    fitted_data = []

    for i in tqdm(range(n)):

        rand_state = i if fix_random else None
        train_data_wide, test_data_wide = get_subsamples(data_wide, train_size=train_size, shuffle=True, random_state=rand_state)

        train = to_long_format(train_data_wide)
        test = to_long_format(test_data_wide)

        p_value, J_st, model = compare_mean(train, test)

        results.append({'p_value': p_value,'J_st': J_st})
        fitted_data.append((model, train, test))

    return pd.DataFrame(results), fitted_data

### Predictions of Mean

Perform n splits and average $J_{st}$ to calculate pooled (bootstrap) p-value\
However, as $J_i$ are correlated as drawn from one sample, CLT does not apply. For this reason we will use Nadeau & Bengio t-test:

$\rho \approx \text{test size} $

$
\widehat{\mathrm{Var}}(\bar J) = (\frac{1}{n} + \frac{\rho}{1 - \rho}) \times \widehat{\mathrm{Var}}(J_i) \quad\text{so that}\quad t = \frac{\bar{J}}{\sqrt{\widehat{\mathrm{Var}}( \bar{J} )}}\;\overset{d}{\to}\; t_{n-1}.
$

Source: [Nadeau, C., & Bengio, Y. (2003). Inference for the Generalization Error. Advances in Neural Information Processing Systems, 16, 307–313](https://proceedings.neurips.cc/paper_files/paper/1999/file/7d12b66d3df6af8d429c1a357d8b9e1a-Paper.pdf)

In [ ]:
### Takes a while to run
n = 100
train_size=0.9
results_mean, fitted_data_mean = cross_val_mean(R, n=n, fix_random=True, train_size=train_size)
results_mean.head(5)

100%|██████████| 100/100 [13:41<00:00,  8.21s/it]


,p_value,J_st
0,0.250506,0.672898
1,0.477570,0.056253
2,0.256952,0.652770
3,0.892014,-1.237309
4,0.100437,1.279064


In [72]:
def NB_test(J, train_size, n):

    J_bar = np.mean(J)
    rho = 1 - train_size

    # Corrected variance: Var(J_bar) = (1/n + rho / (1 - rho)) * Var(J_i)
    s2 = np.var(J, ddof=1)
    var_corrected = (1/n + rho / (1 - rho)) * s2
    se_corrected = np.sqrt(var_corrected)

    t_stat = J_bar / se_corrected
    p_value = 1 - stats.t.cdf(t_stat, df=n-1)  # one-sided test

    return p_value

In [ ]:
J_mean = results_mean['J_st'].values

p_value_mean = NB_test(J_mean, train_size, n)

print('pooled p-value:', p_value_mean)
print('share of positive J_st (unrestricted model performed better):', np.mean(J_mean > 0))

pooled p-value: 0.049791430969976735
share of positive J_st (unrestricted model performed better): 0.73


### Predictions of Variance

Perform the same test on squared residuals, for each split:
1. Estimate $\hat u_{it} = R_{it} - \hat\beta - \hat\lambda_i - \hat\gamma_t $ for train subsample
2. Estimate $\hat v_{jt} = R_{jt} - \hat\beta - \hat\gamma_t $ for test subsample
3. Based on $\hat u^2_{it}$ estimate $\hat k_i$ and $\hat\sigma_t^2$
4. Assess the predictive power of the unrestricted model: $Var(u_{it}) = k_i \times \sigma_t^2$ compared to the restricted model: $Var(u_{it}) = k_i$, based on the test subsample (where $k_i$ is unknown) and thus $\widehat{Var}(v_{jt}) = \hat k \times \hat \sigma_t^2$ vs $\widehat{Var}(v_{jt}) = \hat k$

In [63]:
def cross_val_variance(data):
    """
    Perform cross-validation of the means.
    """

    def get_resid(data_, model_mean):
        
        data = data_.copy()
        gamma = model_mean.estimated_effects.groupby('t').mean()['estimated_effects']
        data['gamma'] = data.index.get_level_values('t').map(gamma)
        data['pred'] = data['gamma'] + model_mean.params['const']
        data['resid'] = data['ret'] - data['pred']

        return data['resid']

    results = []

    for model_mean, train_mean, test_mean in tqdm(data):

        resid_train = get_resid(train_mean, model_mean)

        resid_test = get_resid(test_mean, model_mean)
        test_var = resid_test ** 2

        _, sigma_train = estimate_k_sigma(resid_train.reset_index(), silent = True)
        
        k_restricted = test_var.mean()
        k_unrestricted = (test_var / sigma_train).mean()

        e2_restricted = (test_var - k_restricted) ** 2
        e2_unrestricted = (test_var - k_unrestricted * sigma_train) ** 2

        L = (e2_restricted - e2_unrestricted).groupby('t').mean()
        n = test_var.index.get_level_values('i').nunique()
        
        p_value, J_st = pooled_DM(L, n)
        
        results.append({'p_value': p_value,'J_st': J_st})
        
    return pd.DataFrame(results)

In [65]:
results_var = cross_val_variance(fitted_data_mean).head(5)
results_var.head(5)

100%|██████████| 100/100 [01:05<00:00,  1.53it/s]


,p_value,J_st
0,0.118090,1.184588
1,0.165494,0.972127
2,0.106280,1.246559
3,0.153295,1.022404
4,0.125364,1.148585


In [73]:
J_var = results_var['J_st'].values

p_value_var = NB_test(J_var, train_size, n)

print('pooled p-value:', p_value_var)
print('share of positive J_st (unrestricted model performed better):', np.mean(J_var > 0))

pooled p-value: 0.0
share of positive J_st (unrestricted model performed better): 1.0
